### Arnoldi iterations

This project deals with Krylov subspaces. They are used to find approximate solutions for high dimension linear algebra problems, notably useful in big data. Reduction is a key problem to reduce time computing for very large matrices pow algorithms. Finding eigenvalues and eigenvectors of those very large matrices is the key solution however it is not a simple task.

Arnoldi is analogue to Graham-Schmidt algorithm for similarity transformations to Hessenberg form (nearly triangular matrix). Similarly it can be stopped, leaving with a partial reduction of this form with A a m x m matrix, Q a nearly orthogonal matrix, H the Hessenberg form matrix.

\begin{equation}
A = Q H Q^{*}
\end{equation}

Let's tackle partial reduction with n first columns of AQ = QH with Qn the m x n matrix with  first n columns of Q. H in the next equation correspond to the upper-left (n+1) x n matrix extracted from H.

\begin{equation}
A Q_n = Q_{n+1} \tilde{H_n}
\end{equation}

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
research_link_small = "https://www.cs.cmu.edu/afs/cs/academic/class/15859n-f16/Handouts/TrefethenBau/ArnoldiIteration-33.pdf"
research_link_p_264 = "http://mezbanhabibi.ir/wp-content/uploads/2020/01/NumericalLinearAlgebra-Lloyd-N1.-Trefethen-David-Bau.pdf"

In [3]:
def arnoldi(A,iterations):
    m = A[0].size
    Qn = np.zeros((m,iterations + 1)) 
    H = np.zeros((iterations + 1, iterations))
    b = np.random.randn(m)
    Qn[:,0] = b / np.linalg.norm(b)
    eps = 1e12
    for n in range(iterations):
        v = A @ Qn[:,n]
        for j in range(n + 1):
            H[j,n] = Qn[:,j].conj().T @ v
            v -= H[j,n] * Qn[:,j]
        H[n+1,n] = np.linalg.norm(v)
        if H[n+1,n] > eps : 
            Qn[:,n+1] = v / H[n+1,n]  
        else: 
            return Qn, H
    Q = Qn[:, :iterations]
    H = H[:iterations, :]
    return Q,H

Qn columns {qj} form orthonormal bases for successive Krylov subspaces
H is the upper Hessenberg matrice

In [4]:
m = 6
A = np.random.randint(0,100,(m,m))
Q, H = arnoldi(A,m)
arnoldi_result = (Q @ H @ Q.conj().T).astype(int)
print(arnoldi_result == A, '\n')
print(A, '\n')
print(arnoldi_result)

[[ True False False False False  True]
 [ True False  True False False  True]
 [ True False False False False  True]
 [ True  True False False False  True]
 [ True False False False False  True]
 [ True  True  True False False  True]] 

[[21 81 79 61 98 82]
 [92 20  6 61 47  8]
 [43 66 51 90 89 26]
 [ 0 96 30 12 86 50]
 [13 48 37 26 58 15]
 [59 83  4 12 16 32]] 

[[21 80 78 60 97 82]
 [92 19  6 60 46  8]
 [43 65 50 89 88 26]
 [ 0 96 29 11 85 50]
 [13 47 36 25 57 15]
 [59 83  4 11 15 32]]


In [6]:
def Graham_Schmidt(A):
    m,n = A.shape
    v = np.zeros((m,n))
    r, q = np.zeros((m,n)), np.zeros((m,n))
    for i in range(n):
        v[:,i] = A[:,i]
    for i in range(n):
        r[i,i] = np.linalg.norm(v[:,i])
        q[:,i] = v[:,i] / r[i,i]
        for j in range(i + 1, n):
            r[i,j] = q[:,i].conj().T @ v[:,j]
            v[:,j] -= r[i,j] * q[:,i]
            
    return q, r
        
        

m = 10
A = np.random.randint(0,100,(m,m))
q,r = Graham_Schmidt(A)

print(A, '\n')
print((q @ r).astype(int))

[[80 98  7  7 82 62 62 38 94 30]
 [41 50 13 46 73 88  6 64 11 91]
 [62 31 27 91 32  9 82 41 65 53]
 [89 40 56  6 81 34  3 30 10 54]
 [30 29 84 95 62 87  7  7  7 83]
 [32 35 89 94 57  2 79 72 26 25]
 [64 28 58 82 73 30 10 12  5 59]
 [70 48 69 86 62 51  2 44 35 16]
 [10 61  3 17 47 72 96 10 52 99]
 [ 5 18 56 89 51 80 18 44 32 90]] 

[[80 98  7  7 82 62 62 38 94 30]
 [41 50 12 46 73 88  5 63 11 90]
 [62 31 27 91 32  9 82 40 65 53]
 [89 40 56  6 81 34  3 30 10 54]
 [30 29 84 94 61 87  7  6  7 83]
 [32 35 89 93 57  2 79 72 25 25]
 [64 28 58 82 73 30  9 12  4 58]
 [70 48 69 86 62 51  1 44 35 16]
 [10 60  3 16 47 71 96 10 52 99]
 [ 5 18 55 89 51 80 18 44 32 90]]


## Compute eigenvalues with Arnoldi iteration

At each regular step, the eigenvalues of Hn matrix is computer with QR algorithm (Graham-Schmidt algorithm implemented). Thoses eigenvalues are called "Arnoldi estimates" or "Ritz values". 
Not all eigenvalues are computer because n << dim(A) however extreme eigenvalues (near the edge of spectrum of A). Those are the most useful ones for most applications.



In [ ]:
def lemniscates(A,iterations):
    m,n = A.shape
    b = np.ones(m)
    Q,H = arnoldi(A,iterations)
    p_coeffs = np.poly(H)
    p = np.poly1d(p_coeffs)

    pA = 0
    power = p_coeffs.size - 1
    for coeff in p_coeffs:
        print(pA)
        pA += coeff*np.linalg.matrix_power(A,power)
        power -= 1

    C = np.linalg.norm(pA @ b) / p.linalg.norm(b)
    print("C : ", C)